In [1]:
#import dependencies
import numpy as np
import pandas as pd
import os
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Text, ForeignKey

In [2]:
#Use Pandas to read your cleaned measurements and stations CSV data.
csvfile = "clean_hawaii_measurements.csv"
clean_measurements = pd.read_csv(csvfile, dtype=object)
clean_measurements.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0,63
2,USC00519397,2010-01-03,0,74
3,USC00519397,2010-01-04,0,76
4,USC00519397,2010-01-07,0.06,70


In [3]:
csvfile1 = "clean_hawaii_stations.csv"
clean_stations = pd.read_csv(csvfile1, dtype=object)
clean_stations.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [4]:
#Use the engine and connection string to create a database called hawaii.sqlite.
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
#creating connection to engine
conn = engine.connect()

In [6]:
#Use declarative_base and create ORM classes for each table.
#You will need a class for Measurement and for Station.
#Make sure to define your primary keys.

# Measurement class
Base = declarative_base()

class Measurement(Base):
    __tablename__ = "measurements"
    
    index = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
# Stations class 
class Station(Base):
    __tablename__ = "stations"
    
    station = Column(Text, primary_key=True)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [7]:
# Once you have your ORM classes defined, create the tables in the database using create_all.
Base.metadata.create_all(engine)

In [8]:
# load CSV files into a pandas dataframe
clean_measurements_df = pd.read_csv("clean_hawaii_measurements.csv")
clean_stations_df = pd.read_csv("clean_hawaii_stations.csv")

In [9]:
# using orient=records and creating a list of measurements data 
data = clean_measurements_df.to_dict(orient='records')
data[0]


{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [10]:
#Using Metadata to reflect tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
#creating variable for saving reference to measurements table
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [12]:
#step to remove existing data from table...will comment out before final run
conn.execute(measurements_table.delete())
#insert data into the table
conn.execute(measurements_table.insert(), data)

In [13]:
#checking to be sure insert worked
conn.execute("SELECT * FROM measurements LIMIT 10").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70),
 (6, 'USC00519397', '2010-01-08', 0.0, 64),
 (7, 'USC00519397', '2010-01-09', 0.0, 68),
 (8, 'USC00519397', '2010-01-10', 0.0, 73),
 (9, 'USC00519397', '2010-01-11', 0.01, 64),
 (10, 'USC00519397', '2010-01-12', 0.0, 61)]

In [14]:
#doing same orient=records for stations data
data = clean_stations_df.to_dict(orient='records')
data[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [15]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [16]:
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [17]:
conn.execute(stations_table.delete())
conn.execute(stations_table.insert(), data)

In [18]:
#check to make sure that one worked
conn.execute("SELECT * FROM stations LIMIT 10").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]